# COGS 108 - Data Checkpoint

# Names

- Soham Bhanvadia
- Eric Duong
- Amogh Vig
- Joshua Zhu

<a id='research_question'></a>
# Research Question

Does the frequency of sport injuries in the NBA increase depending on a player's position or role?

# Dataset(s)

(Copy this information for each dataset)
<br>
Dataset Name: <br>
- Link to the dataset:<br>
- Number of observations:<br>
- 1-2 sentences describing each dataset.<br>

If you plan to use multiple datasets, add 1-2 sentences about how you plan to combine these datasets.

Dataset Name: NBA Injuries from 2010-2020 <br>
Link to the dataset: https://www.kaggle.com/datasets/ghopkins/nba-injuries-2010-2018?resource=download<br>
Number of observations: 27105<br>

This dataset includes information about injuries in the NBA from 2010 to 2020. This includes the date of the received injury, the NBA player name, the team, when they recover from the injury (acquired), when they get the injury (relinquished), and any extra notes. This dataset has a new observation for each update on a given athlete's injury, so each athlete may show up on different observation rows multiple times for the same injury time period. However, this dataset does not contain information about position and physical data.

Dataset Name: NBA Height and Weight Analysis<br>
Link to the dataset: https://www.kaggle.com/code/justinas/nba-height-and-weight-analysis/data?select=all_seasons.csv<br>
Number of observations: 11145<br>

We plan on using a second dataset to make up for the lack of information about position and physical data. This dataset includes player information including name, team name, age, height, weight, college, country, draft year, draft round, draft number, games played, points, rebounds, assists, net rating, offensive rebound percent, defensive rebound percent, usage percent, true shooting percent, assist percent, and season played. For our purposes, we only care about athlete data about their names, team name, age, height, weight, games played, and season played.

Dataset Name: NBA Player Statistics<br>
Link to the dataset: https://data.world/etocco/nba-player-stats<br>
Number of observations: 14573<br>

The last dataset we use will include the players' positions. While this dataset includes player statistics such as goals, field goals, three pointers, free throws, and other information about player performance from 1997 to 2022, we only need information about their positions from 2010 to 2020.

We want to combine these datasets with priority placed onto the injury dataset. As the injury dataset contains data from 2010 to 2020, and the player stats datasets contains data from 1997 to 2022, we must merge the data from 2010 to 2020, and only merge together player data that exist on the injury dataset. By matching by name, we can merge together the datasets to include accurate data about each player's position, team, injury, notes, age, height, weight, games played, and seasons played.

# Setup

In [1]:
# import necessary libraries
import numpy as np
import pandas as pd
import math
import matplotlib.pyplot as plt
import seaborn as sns

In [2]:
df_injuries = pd.read_csv('data/injuries_2010-2020.csv')
df_players = pd.read_csv('data/player_stats_1997-2019.csv')
df_teams = pd.read_csv('data/NBA_player_stats.csv')

In [3]:
df_injuries.shape

(27105, 5)

In [4]:
df_injuries.head()

,Date,Team,Acquired,Relinquished,Notes
0,2010-10-03,Bulls,NaN,Carlos Boozer,fractured bone in right pinky finger (out inde...
1,2010-10-06,Pistons,NaN,Jonas Jerebko,torn right Achilles tendon (out indefinitely)
2,2010-10-06,Pistons,NaN,Terrico White,broken fifth metatarsal in right foot (out ind...
3,2010-10-08,Blazers,NaN,Jeff Ayres,torn ACL in right knee (out indefinitely)
4,2010-10-08,Nets,NaN,Troy Murphy,strained lower back (out indefinitely)


In [5]:
df_players.shape

(11145, 22)

In [6]:
df_players.head()

,Unnamed: 0,player_name,team_abbreviation,age,player_height,player_weight,college,country,draft_year,draft_round,...,pts,reb,ast,net_rating,oreb_pct,dreb_pct,usg_pct,ts_pct,ast_pct,season
0,0,Dennis Rodman,CHI,36.0,198.12,99.790240,Southeastern Oklahoma State,USA,1986,2,...,5.7,16.1,3.1,16.1,0.186,0.323,0.100,0.479,0.113,1996-97
1,1,Dwayne Schintzius,LAC,28.0,215.90,117.933920,Florida,USA,1990,1,...,2.3,1.5,0.3,12.3,0.078,0.151,0.175,0.430,0.048,1996-97
2,2,Earl Cureton,TOR,39.0,205.74,95.254320,Detroit Mercy,USA,1979,3,...,0.8,1.0,0.4,-2.1,0.105,0.102,0.103,0.376,0.148,1996-97
3,3,Ed O'Bannon,DAL,24.0,203.20,100.697424,UCLA,USA,1995,1,...,3.7,2.3,0.6,-8.7,0.060,0.149,0.167,0.399,0.077,1996-97
4,4,Ed Pinckney,MIA,34.0,205.74,108.862080,Villanova,USA,1985,1,...,2.4,2.4,0.2,-11.2,0.109,0.179,0.127,0.611,0.040,1996-97


In [7]:
df_teams.shape

(14573, 31)

In [8]:
df_teams.head()

,Rk,Player,Pos,Age,Tm,G,GS,MP,FG,FGA,...,ORB,DRB,TRB,AST,STL,BLK,TOV,PF,PTS,Year
0,1,Mahmoud Abdul-Rauf,PG,28,SAC,31,0,17.1,3.3,8.8,...,0.2,1.0,1.2,1.9,0.5,0.0,0.6,1.0,7.3,1997-1998
1,2,Tariq Abdul-Wahad,SG,23,SAC,59,16,16.3,2.4,6.1,...,0.7,1.2,2.0,0.9,0.6,0.2,1.1,1.4,6.4,1997-1998
2,3,Shareef Abdur-Rahim,SF,21,VAN,82,82,36.0,8.0,16.4,...,2.8,4.3,7.1,2.6,1.1,0.9,3.1,2.5,22.3,1997-1998
3,4,Cory Alexander,PG,24,TOT,60,22,21.6,2.9,6.7,...,0.3,2.2,2.4,3.5,1.2,0.2,1.9,1.6,8.1,1997-1998
4,4,Cory Alexander,PG,24,SAS,37,3,13.5,1.6,3.9,...,0.2,1.1,1.3,1.9,0.7,0.1,1.3,1.4,4.5,1997-1998


# Data Cleaning

After all of the datasets and required packages are loaded, we first need to make all of the column names as well as the 
all of the values on the "notes" column lowercase. This is to ensure consistency between all of the column names and to prevent mismatches due to case sensitivity later when we perform comparisons between "notes" for each row.

In [9]:
#Make all column names lowercase
df_injuries.columns = df_injuries.columns.str.lower()

#Changing all "notes" to be lowercase
df_injuries["notes"] = df_injuries["notes"].apply(str.lower)

df_injuries.head()

,date,team,acquired,relinquished,notes
0,2010-10-03,Bulls,NaN,Carlos Boozer,fractured bone in right pinky finger (out inde...
1,2010-10-06,Pistons,NaN,Jonas Jerebko,torn right achilles tendon (out indefinitely)
2,2010-10-06,Pistons,NaN,Terrico White,broken fifth metatarsal in right foot (out ind...
3,2010-10-08,Blazers,NaN,Jeff Ayres,torn acl in right knee (out indefinitely)
4,2010-10-08,Nets,NaN,Troy Murphy,strained lower back (out indefinitely)


As we see above, there are 5 columns within our original injuries dataset. One of the columns, "acquired", denotes whenever a player returns after recovering from an injury. Although this would be useful for analyzing time missed per each injury, it is often inconsistent depending on the type of injury and contains information outside of the scope of this project. Thus, we will keep all rows not containing "acquired" and remove the column entirely afterwards. This allows us to rename the "relinquished" column to "player", making it more representative of our data.

Furthermore, some players recover/sustain injuries while not contracted to a team, resulting in having no "team" for the respective injury data in the dataset. To prevent inconsistencies in future comparisons, we replace the missing values with "None".

In [10]:
#Keep rows in df that do not have "acquired" value and remove such column afterwards
df_injuries = df_injuries[df_injuries["acquired"].isna() == True]
df_injuries = df_injuries.drop(["acquired"], axis = 1)

#Renaming the "relinquished" column to "player"
df_injuries = df_injuries.rename(columns = {'relinquished': 'player'}) 

#Replacing missing team values
df_injuries["team"] = df_injuries["team"].replace({np.NaN: "None"})
df_injuries.head()

,date,team,player,notes
0,2010-10-03,Bulls,Carlos Boozer,fractured bone in right pinky finger (out inde...
1,2010-10-06,Pistons,Jonas Jerebko,torn right achilles tendon (out indefinitely)
2,2010-10-06,Pistons,Terrico White,broken fifth metatarsal in right foot (out ind...
3,2010-10-08,Blazers,Jeff Ayres,torn acl in right knee (out indefinitely)
4,2010-10-08,Nets,Troy Murphy,strained lower back (out indefinitely)


Finally, before starting to clean up our data, we want to sort the entire dataframe by two columns, "player" and "date". Sorting the data by "player" ensures that for each player, all of their injuries will be grouped together on the dataframe, simplfying our cleaning up process in the future. Similarly, sorting by "date" organizes each players' injuries in chronological order, creating a complete injury history of each player on the dataframe. 

In [11]:
#Converting each date into a datetime object, allowing for comparison between dates
df_injuries['date'] = pd.to_datetime(df_injuries['date'])

#Sort the dataframe by player first, then date
df_injuries = df_injuries.sort_values(by = ['player', 'date'])

#Reset indicies of the rows before cleaning
df_injuries = df_injuries.reset_index(drop = True)
df_injuries.head()

,date,team,player,notes
0,2013-03-01,Hawks,Mike Scott,placed on il
1,2014-11-12,Hawks,Mike Scott,placed on il with back injury
2,2014-12-10,Hawks,Mike Scott,placed on il with flu
3,2015-03-13,Hawks,Mike Scott,placed on il with fractured left big toe
4,2015-04-15,Hawks,Mike Scott,placed on il with back injury


Now, our dataset is ready to be cleaned! One important thing to note is that this dataset not only includes basketball-related injuries, but also any conditions that affect players' abilities to play in games (medical conditions, personal matters, etc.). For example, you can see above that for player "Mike Scott", it is listed that he had the flu on December 10, 2014. Since our project is only focused on basketball-related injuries, we need to first remove all rows that contain injuries/situations for all other designations not related to basketball. 

In [12]:
df_injuries = df_injuries[df_injuries["notes"].str.contains('surgery')==False] #Remove rows with surgery
df_injuries = df_injuries[df_injuries["notes"].str.contains('virus')==False] #Remove rows with virus
df_injuries = df_injuries[df_injuries["notes"].str.contains('flu')==False]  #Remove rows with flu
df_injuries = df_injuries[df_injuries["notes"].str.contains('illness')==False] #Remove rows with illness
df_injuries = df_injuries[df_injuries["notes"].str.contains('infection')==False] #Remove rows with infection
df_injuries = df_injuries[df_injuries["notes"].str.contains('dermatitis')==False] #Remove rows with dermatitis
df_injuries = df_injuries[df_injuries["notes"].str.contains('chicken pox')==False] #Remove rows with chicken pox
df_injuries = df_injuries[df_injuries["notes"].str.contains('stomach ache')==False] #Remove rows with stomach ache
df_injuries = df_injuries[df_injuries["notes"].str.contains('headache')==False] #Remove rows with headache
df_injuries = df_injuries[df_injuries["notes"].str.contains('migraine')==False] #Remove rows with migraine
df_injuries = df_injuries[df_injuries["notes"].str.contains('irregular heartbeat')==False] #Remove rows with irregular heartbeat
df_injuries = df_injuries[df_injuries["notes"].str.contains('heart condition')==False] #Remove rows with heart condition
df_injuries = df_injuries[df_injuries["notes"].str.contains('dizziness')==False] #Remove rows with dizziness
df_injuries = df_injuries[df_injuries["notes"].str.contains('conditioning')==False] #Remove rows with conditioning
df_injuries = df_injuries[df_injuries["notes"].str.contains('rest')==False] #Remove rows with rest
df_injuries = df_injuries[df_injuries["notes"].str.contains('gastroenteritis')==False] #Remove rows with gastroenteritis
df_injuries = df_injuries[df_injuries["notes"].str.contains('food poisoning')==False] #Remove rows with food poisoning
df_injuries = df_injuries[df_injuries["notes"].str.contains('appendectomy')==False] #Remove rows with appendectomy procedures
df_injuries = df_injuries[df_injuries["notes"].str.contains('undisclosed')==False] #Remove rows with undisclosed injuries
df_injuries = df_injuries[df_injuries["notes"].str.contains('fine')==False] #Remove rows with fines

In addition to this, some of these injury designations are not represenative of the injury itself, but rather roster moves of NBA teams related to injuries(i.e. "placed on il (injury list)" means that the player has been moved off the active roster onto the injury list to recover). Since all of these are references to injuries that already are described in the dataset, we can safely remove these rows to trim down our data.

In [13]:
#List of roster moves related to injuries
injury_roster_moves = ["placed on il (out for season)", "dnp", "placed on il (p)", "placed on il (f)", 
                    "placed on il for personal reasons", "placed on il for suspension", "placed on il (f) (out for season)",
                    "dnp (p)", 'placed on il']

#Remove rows that contain such roster moves
df_injuries = df_injuries[df_injuries["notes"].isin(injury_roster_moves) == False]
df_injuries.head()

,date,team,player,notes
1,2014-11-12,Hawks,Mike Scott,placed on il with back injury
3,2015-03-13,Hawks,Mike Scott,placed on il with fractured left big toe
4,2015-04-15,Hawks,Mike Scott,placed on il with back injury
5,2015-12-09,Hawks,Mike Scott,knee injury (dtd)
6,2015-12-10,Hawks,Mike Scott,placed on il with sore knee


In [14]:
df_injuries

,date,team,player,notes
1,2014-11-12,Hawks,Mike Scott,placed on il with back injury
3,2015-03-13,Hawks,Mike Scott,placed on il with fractured left big toe
4,2015-04-15,Hawks,Mike Scott,placed on il with back injury
5,2015-12-09,Hawks,Mike Scott,knee injury (dtd)
6,2015-12-10,Hawks,Mike Scott,placed on il with sore knee
...,...,...,...,...
17537,2019-10-18,Pelicans,Zion Williamson,torn meniscus in right knee (out indefinitely)
17540,2020-02-08,Pelicans,Zion Williamson,sprained left ankle (dtd)
17542,2020-08-10,Pelicans,Zion Williamson,placed on il with sore right knee (out for sea...
17560,2019-03-16,Nets,NaN,activated from il


Here, we have the crux of our dataset. Although the injuries dataset contains all injuries of all NBA players from 2010-2020, this dataset was scraped together by gathering injury reports of all teams from this time period. Since it is required that NBA teams provide reports before or during games regarding players with injuries (unless they are severe injuries with long recovery periods), this results in multiple logs for the same injury across different timespans. This is especially problematic for minor injuries that frequently reoccur, such as sprained ankles or sore knees. We need to clean up these recurring reports for the same injury into a single row, making it so that each row in our dataset represents a single injury for one player.

This is done in 3 main steps. The first is to categorize injuries based upon key words in the "notes" of each row. We can do this by checking for medical terms that indicate both what type of injury it is (i.e. what muscle/bone/tendon etc. is afected) and the region of the injury (i.e. left vs. right ankle). 

In [15]:
#List of key medical terminology indicating the type of injury
injuries_list = ["achilles", "knee", "concussion", "ankle", "back", "hamstring", "quadricep", "hip",
                 "wrist",  "neck", "calf", "foot", "hand", "thumb", "elbow", "shoulder", "groin", 
                 "finger", "illness", "leg", "thigh", "patella", "heel","toe", "tricep", 
                 "bicep", "shin", "rotator", "toenail", "tailbone", "abdominal","groin", 
                 "quadricep", "pectoral", "oblique", "fibula", "tibia", "orbital", "rib", "throat",
                "midfoot", "eye", "cheekbone", "bone", "forearm", "nose", "adductor", "head", "forehead",
                "chest", "pelvis", "arm", "core", "sternum", "hernia", "arm", "stomach", "muscle", 
                 "abductor", "mouth", "cervical", "tooth", "gluteus", "pelvic", "jaw", "cornea", 
                 "nerve", "lat", "glute", "spinal", "quadricap", "lumbar", "spine", "abdomen"]

#List of words indicating region of injury 
regions = ['left', 'right', 'middle', 'lower', 'upper']

#Function that categorizes each injury and adds the type and region of injury as columns into the dataframe
def categorize_injuries(df,injuries,regions):
    injuries_types = []
    injuries_regions = []
    il_list = []
    
    for i in range(0,df.shape[0]): #Loop over each row in the data
        injury_type = ""
        injury_region = ""
        injury_notes = df.iloc[i]['notes'] #Retrive the "notes" of the injury
        
        for injury in injuries:
            if injury in injury_notes: #Check if any key medical injury term is in "notes"
                if len(injury_type) > 0: #Create injury type label 
                    injury_type += "/" + injury
                else:
                    injury_type += injury
        
        for region in regions: 
            if region in injury_notes: #Check if any region labels is in "notes"
                if len(injury_region) > 0: #Create region label
                    injury_region += "/" + region
                else:
                    injury_region += region
        
        if not injury_type: #Assign "None" injury label i unable to classify type of injury
            injury_type = "None"
        if not injury_region: #Assign "None" region label i unable to classify region of injury
            injury_region = "None"
            
        injuries_types.append(injury_type) #Add injury type label
        injuries_regions.append(injury_region) #Add region of injury label
    
    #Create new columns for type and region of injury, add data for each column respectively
    df['type'] = injuries_types 
    df['region'] = injuries_regions
    
    return df

Now we have the type and region for each injury log in the dataset! This gives us a basis for comparison between each injury log and allows us to check whether or not similar injury logs are referring to the same injury, which is the second step. To know if they are from the same injury, there are 3 conditions that need to be met: injury logs contain the same person, have the same injury labels (type and region), and be within the same time-span (less than 20 days). This time component is especially important since players often get the same injury over their careers, and we want to distinguish between these repeat injuries rather than counting them as the same. We selected being within 20 days as the criteria for referencing the same injury, as this is representative of the average recovery time for injuries in the NBA.

In [16]:
#Method checking if the player is the same for subsequent injury logs
def same_player(df, index):
    return df.iloc[index]['player'] == df.iloc[index-1]['player']

#Method checking if the injury labels are the same for subsequent injury logs
def same_injury(df,index):
    return ((df.iloc[index]['type'] == df.iloc[index-1]['type']) & (df.iloc[index]['region'] == df.iloc[index-1]['region']))

#Method checking if the injury logs are within 20 days of each other
def same_time(df,index):
    return (df.iloc[index]['date'] - df.iloc[index-1]['date']).days < 20

Finally, for our third step, we can use these three criteria to remove or keep rows in our dataset. This will allow us to have a dataset where each row represents a single injury (not repeating) per player. We also want to remove rows injuries that were not able to be classified by type since it is likely that such injuries are not basketball-related or baksetball-induced.

In [17]:
#Method for cleaning our dataset by removing redundant injury logs 
def clean_injuries_data(df):
    recurring_rows = []
    
    for i in range(1,df.shape[0]): #Iterate over each row in the dataframe
        if same_player(df,i) and same_injury(df,i) and same_time(df,i): #Compares subsequent injury logs on the three criteria
            recurring_rows.append(i)
    
    df = df.drop(df.index[recurring_rows]) #Drop rows that are classified as logs referencing the same injury
    df = df[df['type'] != "None"] #Remove injuries that could not be classified
    
    df = df.reset_index(drop = True) #Reset the indicies of the rows
    
    return df

With all of our methods and functions defined, we can now call them to clean our dataset!

In [18]:
#Categorizing all injury logs
df_injuries = categorize_injuries(df_injuries, injuries_list, regions)
df_injuries.head(20)
df_injuries = df_injuries.sort_values(by='date')
df_injuries = df_injuries.reset_index(drop=True)

In [19]:
#Cleaning our dataset by removing repeating injury logs 
df_injuries = clean_injuries_data(df_injuries)

In [20]:
df_injuries = df_injuries[['player','team','notes','type', 'region', 'date']]
df_injuries

,player,team,notes,type,region,date
0,Carlos Boozer,Bulls,fractured bone in right pinky finger (out inde...,finger/bone,right,2010-10-03
1,Jonas Jerebko,Pistons,torn right achilles tendon (out indefinitely),achilles,right,2010-10-06
2,Terrico White,Pistons,broken fifth metatarsal in right foot (out ind...,foot,right,2010-10-06
3,Troy Murphy,Nets,strained lower back (out indefinitely),back,lower,2010-10-08
4,Jeff Ayres,Blazers,torn acl in right knee (out indefinitely),knee,right,2010-10-08
...,...,...,...,...,...,...
10986,Romeo Langford,Celtics,placed on il with strained right adductor (out...,adductor,right,2020-09-18
10987,Gabe Vincent,Heat,sore right knee (dtd),knee,right,2020-09-23
10988,Bam Adebayo,Heat,strained left shoulder (dtd),shoulder,left,2020-09-30
10989,Bam Adebayo,Heat,strained neck (dtd),neck,None,2020-10-02


Now, we merge this with the other datasets that we have, starting with height and weight using df_players.

In [21]:
# drop irrelevant columns, rename to fit wanted criteria
df_players = df_players.drop(['Unnamed: 0','team_abbreviation','college', 'country', 'draft_year', 'draft_round','draft_number','pts', 'reb','ast','net_rating','oreb_pct','dreb_pct','usg_pct','ts_pct','ast_pct'], 1)
df_players = df_players.rename(columns={"player_name": "player", "player_height": "height(cm)", "player_weight": "weight(kg)", "gp": "games played"})

In [22]:
df_players

,player,age,height(cm),weight(kg),games played,season
0,Dennis Rodman,36.0,198.12,99.790240,55,1996-97
1,Dwayne Schintzius,28.0,215.90,117.933920,15,1996-97
2,Earl Cureton,39.0,205.74,95.254320,9,1996-97
3,Ed O'Bannon,24.0,203.20,100.697424,64,1996-97
4,Ed Pinckney,34.0,205.74,108.862080,27,1996-97
...,...,...,...,...,...,...
11140,Maxi Kleber,28.0,208.28,108.862080,63,2019-20
11141,Melvin Frazier Jr.,23.0,195.58,97.522280,15,2019-20
11142,Meyers Leonard,28.0,213.36,117.933920,49,2019-20
11143,Norvel Pelle,27.0,208.28,104.779752,20,2019-20


In [23]:
# change season to fit the date format we want (year)
def adjust_season(in_str):
    in_str = str(in_str)
    in_str=in_str.split("-")
    in_str=int(in_str[0])+1
    return in_str

In [24]:
# apply the function
df_players['season'] = df_players['season'].apply(adjust_season)

In [25]:
df_players

,player,age,height(cm),weight(kg),games played,season
0,Dennis Rodman,36.0,198.12,99.790240,55,1997
1,Dwayne Schintzius,28.0,215.90,117.933920,15,1997
2,Earl Cureton,39.0,205.74,95.254320,9,1997
3,Ed O'Bannon,24.0,203.20,100.697424,64,1997
4,Ed Pinckney,34.0,205.74,108.862080,27,1997
...,...,...,...,...,...,...
11140,Maxi Kleber,28.0,208.28,108.862080,63,2020
11141,Melvin Frazier Jr.,23.0,195.58,97.522280,15,2020
11142,Meyers Leonard,28.0,213.36,117.933920,49,2020
11143,Norvel Pelle,27.0,208.28,104.779752,20,2020


In [26]:
# remove players from outside df_injuries' range
df_players = df_players[(df_players['season'] >= 2010) & (df_players['season'] <= 2020)]
df_players = df_players.sort_values(by=['season', 'player'])
df_players = df_players.reset_index(drop=True)

In [27]:
df_players

,player,age,height(cm),weight(kg),games played,season
0,AJ Price,23.0,187.96,82.100152,56,2010
1,Aaron Brooks,25.0,182.88,73.028312,82,2010
2,Aaron Gray,25.0,213.36,122.469840,32,2010
3,Acie Law,25.0,190.50,91.625584,26,2010
4,Adam Morrison,25.0,203.20,92.986360,31,2010
...,...,...,...,...,...,...
5356,Zach LaVine,24.0,198.12,90.718400,60,2020
5357,Zach Norvell Jr.,22.0,195.58,92.986360,5,2020
5358,Zhaire Smith,20.0,190.50,92.986360,7,2020
5359,Zion Williamson,19.0,198.12,128.820128,18,2020


In [28]:
# merge together
df = pd.merge(df_injuries, df_players, on='player', how='inner')

In [29]:
df

,player,team,notes,type,region,date,age,height(cm),weight(kg),games played,season
0,Carlos Boozer,Bulls,fractured bone in right pinky finger (out inde...,finger/bone,right,2010-10-03,28.0,205.74,120.655472,78,2010
1,Carlos Boozer,Bulls,fractured bone in right pinky finger (out inde...,finger/bone,right,2010-10-03,29.0,205.74,120.655472,59,2011
2,Carlos Boozer,Bulls,fractured bone in right pinky finger (out inde...,finger/bone,right,2010-10-03,30.0,205.74,120.655472,66,2012
3,Carlos Boozer,Bulls,fractured bone in right pinky finger (out inde...,finger/bone,right,2010-10-03,31.0,205.74,120.655472,79,2013
4,Carlos Boozer,Bulls,fractured bone in right pinky finger (out inde...,finger/bone,right,2010-10-03,32.0,205.74,120.655472,76,2014
...,...,...,...,...,...,...,...,...,...,...,...
71198,Chris Chiozza,Nets,right adductor injury (dtd),adductor,right,2020-08-13,24.0,180.34,79.378600,19,2020
71199,Donta Hall,Nets,sprained right ankle (dtd),ankle,right,2020-08-13,22.0,205.74,103.872568,3,2020
71200,Chris Silva,Heat,"placed on il with pubic bone injury, strained ...",groin/groin/bone,left,2020-08-18,23.0,203.20,106.140528,40,2020
71201,Vic Law,Magic,placed on il with left achilles injury,achilles,left,2020-08-24,24.0,200.66,91.171992,4,2020


In [30]:
df = df.drop_duplicates(subset=['date'])
df = df.reset_index(drop=True)

In [31]:
df.shape

(1799, 11)

Finally, we merge with the dataset containing the player positions using df_teams.

In [32]:
# make lowercase
df_teams.columns= df_teams.columns.str.strip().str.lower()

In [33]:
# change year format to year, remove times from outside range
df_teams['year'] = df_teams['year'].apply(adjust_season)
df_teams = df_teams[(df_teams['year'] >= 2010) & (df_teams['year'] <= 2020)]

In [34]:
# only include data about player position
df_teams = df_teams[['player', 'pos', 'year']]

In [35]:
# clean up
df_teams = df_teams.drop_duplicates(subset=['player'])
df_teams = df_teams.reset_index(drop=True)

In [36]:
df_teams

,player,pos,year
0,Arron Afflalo,SG,2010
1,Alexis Ajinça,C,2010
2,LaMarcus Aldridge,PF,2010
3,Joe Alexander,SF,2010
4,Malik Allen,PF,2010
...,...,...,...
1339,Coby White,PG,2020
1340,Grant Williams,PF,2020
1341,Nigel Williams-Goss,PG,2020
1342,Zion Williamson,PF,2020


In [37]:
# merge together
df = pd.merge(df, df_teams, on='player', how='left')

In [38]:
df

,player,team,notes,type,region,date,age,height(cm),weight(kg),games played,season,pos,year
0,Carlos Boozer,Bulls,fractured bone in right pinky finger (out inde...,finger/bone,right,2010-10-03,28.0,205.74,120.655472,78,2010,PF,2010.0
1,Carlos Boozer,Bulls,placed on il with fractured bone in right pink...,finger/bone,right,2010-10-27,28.0,205.74,120.655472,78,2010,PF,2010.0
2,Carlos Boozer,Bulls,sprained left ankle (dnp),ankle,left,2011-01-17,28.0,205.74,120.655472,78,2010,PF,2010.0
3,Carlos Boozer,Bulls,sprained left ankle (dnp),ankle,left,2011-01-20,28.0,205.74,120.655472,78,2010,PF,2010.0
4,Carlos Boozer,Bulls,placed on il with sprained left ankle,ankle,left,2011-03-11,28.0,205.74,120.655472,78,2010,PF,2010.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...
1794,Romeo Langford,Celtics,placed on il with strained right adductor (out...,adductor,right,2020-09-18,20.0,193.04,97.975872,25,2020,SG,2020.0
1795,Zach Collins,Blazers,left ankle injury (out for season),ankle,left,2020-08-17,20.0,213.36,104.326160,66,2018,C,2018.0
1796,Dewan Hernandez,Raptors,placed on il with sprained right ankle,ankle,right,2020-03-05,23.0,208.28,106.594120,4,2020,C,2020.0
1797,Gabe Vincent,Heat,sore right knee (dtd),knee,right,2020-09-23,23.0,190.50,90.718400,5,2020,PG,2020.0


In [39]:
# rename columns
df = df[['player', 'team', 'pos', 'notes','type', 'region', 'date', 'age', 'height(cm)', 'weight(kg)', 'games played', 'season']]

In [40]:
# done!
df

,player,team,pos,notes,type,region,date,age,height(cm),weight(kg),games played,season
0,Carlos Boozer,Bulls,PF,fractured bone in right pinky finger (out inde...,finger/bone,right,2010-10-03,28.0,205.74,120.655472,78,2010
1,Carlos Boozer,Bulls,PF,placed on il with fractured bone in right pink...,finger/bone,right,2010-10-27,28.0,205.74,120.655472,78,2010
2,Carlos Boozer,Bulls,PF,sprained left ankle (dnp),ankle,left,2011-01-17,28.0,205.74,120.655472,78,2010
3,Carlos Boozer,Bulls,PF,sprained left ankle (dnp),ankle,left,2011-01-20,28.0,205.74,120.655472,78,2010
4,Carlos Boozer,Bulls,PF,placed on il with sprained left ankle,ankle,left,2011-03-11,28.0,205.74,120.655472,78,2010
...,...,...,...,...,...,...,...,...,...,...,...,...
1794,Romeo Langford,Celtics,SG,placed on il with strained right adductor (out...,adductor,right,2020-09-18,20.0,193.04,97.975872,25,2020
1795,Zach Collins,Blazers,C,left ankle injury (out for season),ankle,left,2020-08-17,20.0,213.36,104.326160,66,2018
1796,Dewan Hernandez,Raptors,C,placed on il with sprained right ankle,ankle,right,2020-03-05,23.0,208.28,106.594120,4,2020
1797,Gabe Vincent,Heat,PG,sore right knee (dtd),knee,right,2020-09-23,23.0,190.50,90.718400,5,2020


Now, all injuries are matched with position played, team name, and athlete physical information!